ingestion airlines

In [3]:
from pyspark.sql import SparkSession

# 1. Création de la session Spark avec support du warehouse
spark = SparkSession.builder \
    .appName("AirlinesCSVtoParquet") \
    .config("spark.sql.warehouse.dir", "spark-warehouse") \
    .enableHiveSupport() \
    .getOrCreate()

# 2. Chargement du fichier CSV
df_airlines = spark.read.option("header", True).csv("Bronze/airlines/*.csv")

# 3. Persistance en mémoire
df_airlines.persist()

# 4. Enregistrement dans le warehouse Spark (sous forme de table)
df_airlines.write.mode("overwrite").saveAsTable("airlines")

# 5. Enregistrement en format Parquet dans le dossier silver
df_airlines.write.mode("overwrite").parquet("silver/airlines")

# Pour vérification (facultatif)
df_airlines.show(5)


+---------+--------------------+
|IATA_CODE|             AIRLINE|
+---------+--------------------+
|       UA|United Air Lines ...|
|       AA|American Airlines...|
|       US|     US Airways Inc.|
|       F9|Frontier Airlines...|
|       B6|     JetBlue Airways|
+---------+--------------------+
only showing top 5 rows



ingestion airports

In [ ]:
# from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# 1. Création de la session Spark avec support du warehouse
# spark = SparkSession.builder \
#     .appName("AirportsCSVtoParquet") \
#     .config("spark.sql.warehouse.dir", "spark-warehouse") \
#     .enableHiveSupport() \
#     .getOrCreate()

# 2. Chargement du fichier CSV
df_airports = spark.read.option("header", True).csv("Bronze/airports/*.csv")



# 3. Persistance en mémoire
# df_airports.persist()

# # 4. Enregistrement dans le warehouse Spark (sous forme de table)
# df_airports.write.mode("overwrite").saveAsTable("airports")

# # 5. Enregistrement en format Parquet dans le dossier silver
# df_airports.write.mode("overwrite").parquet("silver/airports")

# Pour vérification (facultatif)
df_airports.show(5)


+---------+--------------------+-----------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|       CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+--------------------+-----------+-----+-------+--------+----------+
|      ABE|Lehigh Valley Int...|  Allentown|   PA|    USA|40.65236| -75.44040|
|      ABI|Abilene Regional ...|    Abilene|   TX|    USA|32.41132| -99.68190|
|      ABQ|Albuquerque Inter...|Albuquerque|   NM|    USA|35.04022|-106.60919|
|      ABR|Aberdeen Regional...|   Aberdeen|   SD|    USA|45.44906| -98.42183|
|      ABY|Southwest Georgia...|     Albany|   GA|    USA|31.53552| -84.19447|
+---------+--------------------+-----------+-----+-------+--------+----------+
only showing top 5 rows



ingestion flights

In [11]:
from pyspark.sql import SparkSession

# 1. Créer la session Spark
spark = SparkSession.builder \
    .appName("ConcatCSVtoParquet") \
    .getOrCreate()

# 2. Lire tous les fichiers CSV dans le dossier Bronze/
df = spark.read.option("header", True).csv("Bronze/*.csv")

threshold = 0.4
row_count = df.count()

df_cleaned = df.select([
    c for c in df .columns
    if df.filter(col(c).isNotNull()).count() / row_count >= (1 - threshold)
])

# 3. Écrire tous les fichiers fusionnés dans Silver/ en format Parquet
df_cleaned.write.mode("overwrite").parquet("Silver/flights_table")

# (Optionnel) Afficher quelques lignes pour vérifier
df_cleaned.show(100)


+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+--------+-----------------+--------+---------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|SCHEDULED_TIME|DISTANCE|SCHEDULED_ARRIVAL|DIVERTED|CANCELLED|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+--------+-----------------+--------+---------+
|2015|   11| 20|          5|     UA|          791|     N462UA|           ORD|                LGA|               1100|           129|     733|             1409|       0|        0|
|2015|   11| 20|          5|     UA|         1120|     N41135|           EWR|                SFO|               1100|           396|    2565|             1436|       0|        0|
|2015|   11| 20|          5|     UA|         1560|     N87531|           MCO|                EWR|        